In [43]:
import numpy as np
import pandas as pd
import pickle

In [50]:
# Import my modules.
import sys, os
from pathlib import Path
current_dir = os.path.join(Path().resolve())
sys.path.append(str(current_dir) + '/../')

from modules import utils
from modules import models
from modules import preprocess

import importlib
for m in [utils, models, preprocess]:
    importlib.reload(m)

In [45]:
pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 150)

In [13]:
mbd = 'microbusiness_density'

df_train, df_test, df_subm = utils.load_dataset(subm='../submission/submission_13730_trend.csv')
df_all, df_census = utils.merge_dataset(df_train, df_test, unemploy=False, census=True, co_est=False, coord=True, add_location=True, 
                                        fix_pop=True, merge41=True, df_subm=df_subm)

merge scale=41 of df_submission to df_all.


In [51]:
params={
        "act_thre": 140,
        "abs_thre": 0,
        "USE_LAG": 8,
        "USE_TREND": True,
        "blacklist": [],
        "blacklistcfips": [],
        "clip": (-0.0044, 0.0046),
        "model": 'lgb',
        "max_window": 12,
        "start_max_scale": 41,
        "start_all_dict": 32,
        "smooth_method": 'v3',
        "save_output_dic": True,
        "light": True,
        "USE_SEASON": True
    }
trend_params = {
        "high_trend_params": {
            1: {
                'params':{
                    'n':3,
                    'thre':3,
                    'thre_r':0,
                    'lower_bound': 20000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.01]
                },
                'method': 'mean'
            },
            2: {
                'params':{
                    'n':4,
                    'thre':4,
                    'thre_r':0.001,
                    'lower_bound': 20000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.01]
                },
                'method': 'mean'
            },
            3: {
                'params':{
                    'n':5,
                    'thre':5,
                    'thre_r':0.002,
                    'lower_bound': 20000,
                    'upper_bound': 99999999,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.01]
                },
                'method': 'mean'
            }
        },
        "low_trend_params": {
            1: {
                'params':{
                    'n':5,
                    'thre':5,
                    'thre_r':0,
                    'lower_bound': 2000,
                    'upper_bound': 20000,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.02]
                },
                'method': 'mean'
            },
            2: {
                'params':{
                    'n':6,
                    'thre':6,
                    'thre_r':0.001,
                    'lower_bound': 2000,
                    'upper_bound': 20000,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.02]
                },
                'method': 'mean'
            },
            3: {
                'params':{
                    'n':7,
                    'thre':7,
                    'thre_r':0.002,
                    'lower_bound': 2000,
                    'upper_bound': 20000,
                    'use_regularize': True,
                    'v_regularize': [0.002, 0.001],
                    'v_clip':[0.98, 1.02]
                },
                'method': 'mean'
            }
        }
}
season_params = {
    "abs_thre": [-0.006, 0.006],
    "active_thre": 5000,
    "v_clip": [-0.01, 0.01],
    "method": "trend_mean"
}

## バリデーション

### 前の予測結果を用いて予測していく

In [ ]:
accum_pred = models.LgbmBaseline('params_accum',df_subm, df_all, df_census, params=params)
accum_pred.accum_validation()

create df_all_dict[32] and df_all_dict_original[32]
add lag features: max_scale=32
smooth_outlier: max_scale=32
used method: v3
# of fixed cfips: 1123
# of fixed value: 2015


## Submissionの作成

### 予測結果を活用して予測

In [38]:
test_subm = models.LgbmBaseline('test_subm_accum', df_subm, df_all, df_census, params=params, trend_params=trend_params)
test_subm.create_submission(target_scale=[42, 43,44,45], save=False)

create df_all_dict[40] and df_all_dict_original[40]
add lag features: max_scale=40
smooth_outlier: max_scale=40
used method: v3
# of fixed cfips: 1189
# of fixed value: 2310
create df_all_dict[41] and df_all_dict_original[41]
add lag features: max_scale=41
smooth_outlier: max_scale=41
used method: v3
# of fixed cfips: 1195
# of fixed value: 2343
create df_season, max_scale: 40, validate: False
valid_times:  42
pred_m:  1
train_times:  41
use df_all_dict[41]
['scale', 'state_i', 'pct_bb_2017', 'pct_bb_2018', 'pct_bb_2019', 'pct_bb_2020', 'pct_bb_2021', 'pct_college_2017', 'pct_college_2018', 'pct_college_2019', 'pct_college_2020', 'pct_college_2021', 'pct_foreign_born_2017', 'pct_foreign_born_2018', 'pct_foreign_born_2019', 'pct_foreign_born_2020', 'pct_foreign_born_2021', 'pct_it_workers_2017', 'pct_it_workers_2018', 'pct_it_workers_2019', 'pct_it_workers_2020', 'pct_it_workers_2021', 'median_hh_inc_2017', 'median_hh_inc_2018', 'median_hh_inc_2019', 'median_hh_inc_2020', 'median_hh_inc

In [9]:
df_submission =test_subm.df_submission
df_merged = utils.compare_submission(df_subm, 'submission_13730_trend')
df_merged['smape'].sum()

0.0

In [8]:
# ensemble
0.01261112205424169

# trend_multi有り
0.03200090765098272

0.03200090765098272

In [24]:
id1 = (df_all['county']==20)|(df_all['county']==40)
id2 = (df_all['scale']==41)&id1

In [26]:
sum(id2)

2